In [49]:
from langchain.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader(
    "docs/test.md", mode="elements", languages=["zh"], detect_language_per_element=True
)
data = loader.load()

for doc in data:
    print(doc)
    print(f"【cat: {doc.metadata['category']}】\n")
    print(doc.page_content[:100])
    print("---")


page_content='例4.家庭中之父母於裝置上為其子女建立帳戶，選擇並管理子女所使用的PIN或通行碼。父母係裝置管理者，可限制子女變更PIN或通行碼。備考1. 預期製造者將實施適當維護產品中使用之所有軟體及硬體，此包括適當維護及提供相關聯服務所選定的第三方，以支援產品功能。' metadata={'source': 'docs/test.md', 'last_modified': '2024-07-15T23:40:04', 'languages': ['kor'], 'filetype': 'text/markdown', 'file_directory': 'docs', 'filename': 'test.md', 'category': 'UncategorizedText'}
【cat: UncategorizedText】

例4.家庭中之父母於裝置上為其子女建立帳戶，選擇並管理子女所使用的PIN或通行碼。父母係裝置管理者，可限制子女變更PIN或通行碼。備考1. 預期製造者將實施適當維護產品中使用之所有軟體及硬體，此包括適
---
page_content='軟體解決方案通常包含開放原始碼及第三方軟體組件。建立並維護所有軟體組件及其子組件之列表，係能監視產品脆弱性的先決條件。存在各種工具用以掃描原始碼及二進碼，並構建所謂軟體組成清單(SBOM)，其可識別第三方組件及產品中所使用的版本。然後，此資訊用以監視各已識別軟體組件之相關聯安全及使用授權的風險。' metadata={'source': 'docs/test.md', 'last_modified': '2024-07-15T23:40:04', 'languages': ['kor'], 'filetype': 'text/markdown', 'file_directory': 'docs', 'filename': 'test.md', 'category': 'UncategorizedText'}
【cat: UncategorizedText】

軟體解決方案通常包含開放原始碼及第三方軟體組件。建立並維護所有軟體組件及其子組件之列表，係能監視產品脆弱性的先決條件。存在各種工具用以掃描原始碼及二進碼，並構建所謂軟體組成清單(SBOM)，其可識別第
---
page_content=